<a href="https://colab.research.google.com/github/ctandrewtran/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 3: Oil Blending Problem

Group members: Andrew Tran, Ryan Bilas, Elizabeth Gurry, Joshua Allen, Matt Mester

In [357]:
#http://www.producao.ufrgs.br/arquivos/disciplinas/382_winston_cap3_introduction_to_linear_programming.pdf

#http://www.producao.ufrgs.br/arquivos/disciplinas/382_winston_cap3_introduction_to_linear_programming.pdf
# before you do anything...
# mount your drive!
# click folder on the left...

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

## Model, Variable, Constraints

In [426]:
##declaring the model
model = ConcreteModel()

###variables
##variables are the crude stock and its final blend
##there are three sections of variables, with each section representing the final blend
##inside each section, there are 4 variables inside which represent the individual crude stocks

#regular blend
model.cs1_reg = Var(domain=NonNegativeReals)
model.cs2_reg = Var(domain=NonNegativeReals)
model.cs3_reg = Var(domain=NonNegativeReals)
model.cs4_reg = Var(domain=NonNegativeReals)

#multigrade blend
model.cs1_multi = Var(domain=NonNegativeReals)
model.cs2_multi = Var(domain=NonNegativeReals)
model.cs3_multi = Var(domain=NonNegativeReals)
model.cs4_multi = Var(domain=NonNegativeReals)

#supreme blend
model.cs1_sup = Var(domain=NonNegativeReals)
model.cs2_sup = Var(domain=NonNegativeReals)
model.cs3_sup = Var(domain=NonNegativeReals)
model.cs4_sup = Var(domain=NonNegativeReals)


###sales
##sales are defined as the sum of the barrels per blend type (aka variable section seen above) times the selling price of that blend
##we then sum up that workflow for all blends and we arrive at our answer for sales
sales = (model.cs1_reg + model.cs2_reg + model.cs3_reg + model.cs4_reg)*8.5 + (model.cs1_multi + model.cs2_multi + model.cs3_multi + model.cs4_multi)*9 + (model.cs1_sup + model.cs2_sup + model.cs3_sup + model.cs4_sup)*10

###production costs
##prod cost is found in a similar way to sales
##prod cost is defined as the sum of barrels per crude stock type times buying price of that blend
##we then sum up that workflow for all crude stocks and we arrive at our answer for production costs
prod_cost = (model.cs1_reg + model.cs1_multi + model.cs1_sup)*7.1 +(model.cs2_reg + model.cs2_multi + model.cs2_sup)*8.5 + (model.cs3_reg + model.cs3_multi + model.cs3_sup)*7.7 + (model.cs4_reg + model.cs4_multi + model.cs4_sup)*9

###objective to maximize profit per sales minus production costs
model.profit = Objective(expr = sales - prod_cost, sense = maximize)


###viscosity constraints
##found by cs barrels X *viscosity for cs barrels X / cs barrels X amount but rexpressed in order to work with pyomo
model.reg_mix = Constraint(expr = model.cs1_reg*5 + model.cs2_reg*-15 + model.cs3_reg*-5 + model.cs4_reg*-30 <= 0)
model.multi_mix = Constraint(expr = model.cs1_multi*15 + model.cs2_multi*-5 + model.cs3_multi*5 + model.cs4_multi*-20 <= 0)
model.sup_mix = Constraint(expr = model.cs1_sup*30 + model.cs2_sup*10 + model.cs3_sup*20 + model.cs4_sup*-5 <= 0)

###demand constraint
##demand must be met exactly 
##found by sum of barrels per blend
model.reg_dem = Constraint(expr = model.cs1_reg + model.cs2_reg + model.cs3_reg + model.cs4_reg == 2000)
model.multi_dem = Constraint(expr = model.cs1_multi + model.cs2_multi + model.cs3_multi + model.cs4_multi == 1500)
model.sup_dem = Constraint(expr = model.cs1_sup + model.cs2_sup + model.cs3_sup + model.cs4_sup == 750)


###supply constraint
##supply only has a ceiling
##found by summing all barrels per crude stock type
model.cs1_supply = Constraint(expr = model.cs1_reg + model.cs1_multi + model.cs1_sup <= 1000)
model.cs2_supply = Constraint(expr = model.cs2_reg + model.cs2_multi + model.cs2_sup <= 1100)
model.cs3_supply = Constraint(expr = model.cs3_reg + model.cs3_multi + model.cs3_sup <= 1200)
model.cs4_supply = Constraint(expr = model.cs4_reg + model.cs4_multi + model.cs4_sup <= 1100)

### solve!
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 3760.0, 'Upper bound': 3760.0, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 13, 'Number of nonzeros': 37, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.012751340866088867}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Solve!

In [427]:
## code to generate report 
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model)

# Save the model 
model.write("/content/model.lp", io_options={'symbolic_solver_labels': True})

# Generate the file "sensit.sen", this contains the report we want to see
!/usr/bin/glpsol -m /content/model.lp --lp --ranges sensit.sen

# Display the contents of the report: (this shows the sensitivity analysis report)
!cat /content/sensit.sen

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /content/model.lp --lp --ranges sensit.sen
Reading problem data from '/content/model.lp'...
11 rows, 13 columns, 37 non-zeros
100 lines were read
GLPK Simplex Optimizer, v4.65
11 rows, 13 columns, 37 non-zeros
Preprocessing...
10 rows, 12 columns, 36 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+01  ratio =  3.000e+01
GM: min|aij| =  6.389e-01  max|aij| =  1.565e+00  ratio =  2.449e+00
EQ: min|aij| =  4.082e-01  max|aij| =  1.000e+00  ratio =  2.449e+00
Constructing initial basis...
Size of triangular part is 10
      0: obj =   7.500000000e+02 inf =   1.371e+03 (2)
      2: obj =   3.505000000e+03 inf =   0.000e+00 (0)
*     6: obj =   3.760000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (43461 bytes)
Write sensitivity analysis report to 'sensit.sen'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                        

In [428]:
##extra prints for double check

print("CS1 used: " + str(model.cs1_reg() + model.cs1_multi() + model.cs1_sup()))
print("CS2 used: " + str(model.cs2_reg() + model.cs2_multi() + model.cs2_sup()))
print("CS3 used: " + str(model.cs3_reg() + model.cs3_multi() + model.cs3_sup()))
print("CS4 used: " + str(model.cs4_reg() + model.cs4_multi() + model.cs4_sup()))
print(" ")
print("Reg sold: " + str(model.cs1_reg() + model.cs2_reg() + model.cs3_reg() + model.cs4_reg()))
print("Multi sold: " + str(model.cs1_multi() + model.cs2_multi() + model.cs3_multi() + model.cs4_multi()))
print("Sup sold: " + str(model.cs1_sup() + model.cs2_sup() + model.cs3_sup() + model.cs4_sup()))
print(" ")

CS1 used: 1000.0
CS2 used: 1100.0
CS3 used: 1199.9999999999975
CS4 used: 950.0
 
Reg sold: 2000.0000000000007
Multi sold: 1499.9999999999968
Sup sold: 750.0
 
